# Simulation Preliminary Functions

In [4]:
# Import statements
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pandas as pd
import random
from collections import defaultdict
import copy
import matplotlib.colors as c
from matplotlib.cm import hsv
plt.rcParams['axes.axisbelow'] = True

### Read in Experimental Data Files

In [5]:
def read_exp_data(filepath):
    '''
    Reads in an experimental data file for analysis.
    
    PARAMETERS:
    -----------
    filepath: string; path to file in the larger directory configuration.
    
    RETURNS:
    --------
    filedata: Pandas DataFrame; dataframe containing experimental data.
    '''
    
    filedata = pd.read_csv(filepath)
    return filedata

outerfilepath = './datafiles/'

In [6]:
### read in face experiment data
f20h1 = read_exp_data(outerfilepath +'f20h1.csv')
f20h2 = read_exp_data(outerfilepath +'f20h2.csv')
f20h3 = read_exp_data(outerfilepath +'f20h3.csv')

f50h1 = read_exp_data(outerfilepath +'f50h1.csv')
f50h2 = read_exp_data(outerfilepath +'f50h2.csv')
f50h3 = read_exp_data(outerfilepath +'f50h3.csv')

f20s1 = read_exp_data(outerfilepath +'f20s1.csv')
f20s2 = read_exp_data(outerfilepath +'f20s2.csv')
f20s3 = read_exp_data(outerfilepath +'f20s3.csv')

f50s1 = read_exp_data(outerfilepath +'f50s1.csv')
f50s2 = read_exp_data(outerfilepath +'f50s2.csv')
f50s3 = read_exp_data(outerfilepath +'f50s3.csv')

In [7]:
### read in hashtag experiment data
h20h1 = read_exp_data(outerfilepath +'h20h1.csv')
h20h2 = read_exp_data(outerfilepath +'h20h2.csv')
h20h3 = read_exp_data(outerfilepath +'h20h3.csv')

h50h1 = read_exp_data(outerfilepath +'h50h1.csv')
h50h2 = read_exp_data(outerfilepath +'h50h2.csv')
h50h3 = read_exp_data(outerfilepath +'h50h3.csv')

h100h1 = read_exp_data(outerfilepath +'h100h1.csv')

h20s1 = read_exp_data(outerfilepath +'h20s1.csv')
h20s2 = read_exp_data(outerfilepath +'h20s2.csv')
h20s3 = read_exp_data(outerfilepath +'h20s3.csv')

h50s1 = read_exp_data(outerfilepath +'h50s1.csv')
h50s2 = read_exp_data(outerfilepath +'h50s2.csv')
h50s3 = read_exp_data(outerfilepath +'h50s3.csv')

h100s1 = read_exp_data(outerfilepath +'h100s1.csv')

### Need Experimental Priors 

These are generated based on the experimental data.

In [8]:
face_exp_dfs = [f20h1, f20h2, f20h3, f20s1, f20s2, f20s3, f50h1, f50h2, f50h3, f50s1, f50s2, f50s3]
hashtag_exp_dfs = [h20h1, h20h2, h20h3, h20s1, h20s2, h20s3, h50h1, h50h2, h50h3, h50s1, h50s2, h50s3, h100h1, h100s1]

def get_prior(df_lst):
    '''
    Given a list of experiment dataframes, will generate the prior for all of them. i.e. all of the trial one responses.
    
    PARAMETERS:
    -----------
    df_lst: list of Pandas DataFrame objects; all of the experimental data being used to generate the prior
    
    RETURNS:
    --------
    prior: list; a list of every response said on the first trial in all of the given experiments.
    '''
    
    prior = []
    for df in df_lst:
        prior += df[df['TrialNumber'] == 1].Response.tolist() # adds all trial 1 responses to prior
    
    return prior

faceprior = get_prior(face_exp_dfs)
htprior = get_prior(hashtag_exp_dfs)

### Behind-the-Scenes Functions

* get_pairs_from_file
* gen_network
* compute_entropy, compute_avg_entropy
* prob_background, binary_decision
* Plotting functions:
    * decision_type_stacked_bar
    * decision_type_line
    * decision_type_heatmap
    * entropy_line_graph
    * response_heatmap
    * most_common_responses
    * plot_transition_probs_bar
    * points_by_decision_type
    * **create_all_plots**

In [9]:
def get_pairs_from_file(n, network_type, num_rounds):
    '''
    This function returns a list of pairings on the specified network type by reading in the pairing file as a csv and extracting the relevant information as a list.
    This is a helper function that will be called within the gen_network function.
    
    PARAMETERS:
    -----------
    n: integer; number of nodes on the network.
    network_type: string; either spatially embedded or homogeneously mixing.
    num_rounds: integer; number of rounds we want to run the simulation.
    
    RETURNS:
    --------
    pairs_lst: a list of lists of tuples.
    There are num_rounds number of internal lists, and each one of those contains tuples which specify the pairings for that round.
    '''
    filepath = './pairfiles/'
    
    if network_type == "spatially_embedded": # if the network is spatial, want to navigate to spatial files.
        filepath += 'spatial network/'
        neighborsize = 4
    elif network_type == "homogeneously_mixing": # otherwise, we will do homogeneously mixing files.
        filepath += 'homogeneous network/'
        neighborsize = n
    
    seed = random.choice(range(1, 11)) # allows picking of one of the options randomly. There are 10 different files for each circumstance.
    
    file_name = f'connection_{n}.40.{neighborsize}_{seed}.csv' # gives the next part of the file path using relevant information based on the file naming conventions
    
    pairs = pd.read_csv(filepath + file_name, header=None)
    
    pairs_lst = []
    for i in range(0, num_rounds): 
        pairings = [(pairs[0][ind], pairs[1][ind]) for ind in pairs.index if pairs[2][ind] == ((i%40)+1)]
        pairs_lst.append(pairings)
    return pairs_lst

In [10]:
def gen_network(n, network_type, num_rounds):
    '''A helper function that generates a topology to run the simulated experiment on.
       PARAMETERS:
       -----------
       n: integer; number of nodes
       network_type: string; type of network topology wanted. Can be spatially_embedded, random, or homogeneously_mixing
       num_rounds: integer; number of rounds the simulation will run for.
       num_connections: this applies to the spatially embedded and random network models, the desired degree of each node in the network.\
       
       RETURNS:
       --------
       final_G: graph; network graph object with the specified attributes.
       pairs: list; the pairings for the simulation.'''
    
    # make it a dictionary like {1: {'choice': Hannah, 'score': 0, etc.}}
    node_attrs = {i: {'choice': -1,
                      'score': 0, # keeps track of points for each node
                      'memory': []} for i in range(n)}
    # stores all necessary node attributes in a dictionary, can update as necessary
    
    if network_type == "spatially_embedded":
        # generate a graph model where each node is connected to a certain number (4) of the closest nodes in the network
        # uses newman watts strogatz with an edit that raises the degree if degree is odd instead of lowering it
            
        G = nx.newman_watts_strogatz_graph(n, 4, 0)
        nx.set_node_attributes(G, node_attrs)
        
    elif network_type == "homogeneously_mixing":
        # generate a complete graph of degree n
        G = nx.complete_graph(n)
        nx.set_node_attributes(G, node_attrs)
    
    ### no simulation of random networks here... may want to re-add that for final data analysis
    ### this is because there are no files that have random graph pairings
        
    else:
        print("ERROR: Graph type must be valid. please indicate either 'spatially_embedded' or 'homogeneously_mixing'.")
        return
    
    final_G = nx.relabel_nodes(G, lambda x: x+1)
    
    ## MAKE PAIRS
    pairs = get_pairs_from_file(n, network_type, num_rounds)
    
    return final_G, pairs

In [11]:
def compute_entropy(df):
    pivot_df = df.pivot(index='id', columns='TrialNumber', values='Response')
    entropy_values = []

    for column in pivot_df.columns:
        counts = pd.Series(pivot_df[column]).value_counts()
        proportions = counts / counts.sum()
        
        entropy = -np.sum(proportions * np.log2(proportions))
        entropy_values.append(entropy)

    return entropy_values


def compute_avg_entropy(exp_lst):
    delta_entropies = []
    for exp in exp_lst:
        delta_entropies.append(np.diff(np.array(compute_entropy(exp))))
        
    avg_entropy = np.mean(delta_entropies, axis=0)
    return avg_entropy

In [12]:
def prob_background(alpha, T, C): # weight
    '''alpha and beta are user defined parameters
    T: current simulation timestep
    C: number of points gained by sampling from context (RS or RP)'''
    prob_background = alpha/ (T + alpha + C)
    return prob_background

## THIS IS IMPORTANT, ALLOWS FOR DECISION TYPES
def binary_decision(rp_points, context_points, gamma):
    '''
    Generates the probability of switching to partner choice based on percieved utility of RP.
    
    rp_points: integer; total points scored by individual using rp strategy.
    context_points: integer; total points scored by individual using rp or rs strategy. 
    
    gamma: float in (0, 1]; allows adjustment of the probability of RP.
    '''
    prob_rp = gamma*((rp_points+ 1)/(context_points + 2))
    
    return prob_rp

### Functions for Plotting 

In [13]:
def calc_transition_probs(df):
    '''
    Calculates transition probabilities between different decision types given experiment/simulation dataframe.
    
    PARAMETERS:
    -----------
    df: Pandas DataFrame; contains information about an experimental or simulation run.
    
    RETURNS:
    --------
    transition_probs: dictionary; keys are tuples of decision state combinations, values are the transition probabilities from key[0] to key[1].
    '''
    decisiontypes = pd.Series(df.groupby('id')['DecisionType'])

    dtdict = {id: [] for id in df.id.unique()}

    for i in decisiontypes.index:
        if i + 1 in dtdict.keys():
            dtdict[i+1] += decisiontypes[i][1].tolist()
        else:
            dtdict[i+1] = decisiontypes[i][1].tolist()

    newtups = []
    for k in dtdict.keys():
        for i in range(len(dtdict[k]) - 1):
            newtups.append((dtdict[k][i], dtdict[k][i+1]))

    transition_probs = {v: newtups.count(v)/len(newtups) for v in set(newtups)}
    return transition_probs

In [14]:
def decision_type_stacked_bar(df, axis):
    '''
    Creates a stacked bar graph of the decision types given a dataframe.
    
    PARAMETERS:
    -----------
    df: Pandas DataFrame; contains information from a simulation run or experiment to plot.
    axis: matplotlib axis object; the axis on which to plot the data.
    
    RETURNS:
    --------
    None
    '''
    
    categories = ['BN', 'EC', 'RS', 'RP'] # three decision types NEW (pull from prior), RS (repeat self), RP (repeat partner)
    colors = ['xkcd:light red', 'xkcd:kelly green','xkcd:goldenrod', 'xkcd:cerulean'] # define colors for graph
    
    grouped_trial = df.groupby(['TrialNumber', 'DecisionType']).size().unstack(fill_value=0) # group the data by trial and decision type columns for plotting
    grouped_trial = grouped_trial.loc[:, grouped_trial.sum().sort_values(ascending=False).index] # Sort columns by total count

    max_trial = df['TrialNumber'].max()
    # the next two lines ensure that the bars are always plotted in the same order (NEW -> RS -> RP) regardless of value counts
    grouped_trial.columns = pd.CategoricalIndex(grouped_trial.columns.values, ordered=True, categories=categories) 
    grouped_trial = grouped_trial.sort_index(axis=1)

    grouped_trial.plot(kind='bar', stacked=True, color=colors, ax = axis) # creates the stacked bar plot
    
    ## appearance settings: graph and axes titles, ticks, legend.
    axis.set_title('Trial-level Decision Types \n(Stacked Bar)', fontsize=14)
    axis.set_xlabel('Trial (Round)', fontsize=12)
    axis.set_xticks(range(0, max_trial +1, 2))
    axis.tick_params(axis='x', labelrotation = 0) # ensures that xticks are vertically oriented.
    axis.set_ylabel('Number of Participants', fontsize=12)
    handles, labels = axis.get_legend_handles_labels()
    axis.legend(reversed(handles), reversed(labels), title='Decision Type', loc='upper left', bbox_to_anchor=(1, 1)) # this ensures that the legend displays in the order that the bars display

In [15]:
def decision_type_line(df, axis):
    '''
    Creates a line/scatter graph of the decision types given a dataframe.
    
    PARAMETERS:
    -----------
    df: Pandas DataFrame; contains information from a simulation run or experiment to plot.
    axis: matplotlib axis object; the axis on which to plot the data.
    
    RETURNS:
    --------
    None
    '''
    
    categories = ['BN', 'EC','RS', 'RP'] # three decision types NEW (pull from prior), RS (repeat self), RP (repeat partner)
    colors = ['xkcd:light red', 'xkcd:kelly green','xkcd:goldenrod', 'xkcd:cerulean'] # define colors for graph
    
    grouped_trial = df.groupby(['TrialNumber', 'DecisionType']).size().unstack(fill_value=0) # group the data by trial and decision type columns for plotting
    grouped_trial = grouped_trial.loc[:, grouped_trial.sum().sort_values(ascending=False).index] # Sort columns by total count

    max_trial = df['TrialNumber'].max()
    # the next two lines ensure that the bars are always plotted in the same order (NEW -> RS -> RP) regardless of value counts
    grouped_trial.columns = pd.CategoricalIndex(grouped_trial.columns.values, ordered=True, categories=categories) 
    grouped_trial = grouped_trial.sort_index(axis=1)
    
    # A. create a line plot (and scatter values) of the number of agents using BN (Brand New) decision types.
    axis.plot(grouped_trial.index, grouped_trial['BN'], color=colors[0], alpha=0.5)
    axis.scatter(grouped_trial.index, grouped_trial['BN'], color=colors[0], marker='o', s=100, label='BN')
    
    # B. create a line plot and scatter values of agents using EC (Earlier Context) decision types
    axis.plot(grouped_trial.index, grouped_trial['EC'], color=colors[1], alpha=0.5)
    axis.scatter(grouped_trial.index, grouped_trial['EC'], color=colors[1], marker='o', s=100, label='EC')
    
    # C. create a line plot (and scatter values) of the number of agents using RS decision types.
    axis.plot(grouped_trial.index, grouped_trial['RS'], color=colors[2], alpha=0.5)
    axis.scatter(grouped_trial.index, grouped_trial['RS'], color=colors[2], marker='o', s=100, label='RS')
    
    # Create a line plot (and scatter points) of RS=RP
    # if 'old_decisiontype' in df.columns:
    #     rs_rp = df.groupby(['TrialNumber', 'old_decisiontype']).size().unstack(fill_value=0)['RS=RP']
    #     axis.scatter(rs_rp.index, rs_rp, marker='s', s=25, color='magenta', label='Repeat Reward (RS=RP)')
    #     axis.plot(rs_rp, color='magenta', ls='-.', alpha=0.5)
    
    # D. create a line plot (and scatter values) of the number of agents using RP decision types.
    axis.plot(grouped_trial.index, grouped_trial['RP'], color=colors[3], alpha=0.5)
    axis.scatter(grouped_trial.index, grouped_trial['RP'], color=colors[3], marker='o', s=100, label='RP')
    
    # E. create a line plot (and scatter values) of points per trial.
    if 'Points' in df.columns:
        pts_per_trial = df.groupby(['TrialNumber'])['Points'].sum()
        axis.scatter(pts_per_trial.index, pts_per_trial, marker = '^', s = 50, color = 'k', label='POINTS')
        axis.plot(pts_per_trial, color='k', ls='-.', alpha=0.5)

    ## appearance settings: graph and axes titles, ticks, legend.
    axis.set_title('Trial-Level Decision Types\n (Scatter Plot)', fontsize=14)
    axis.set_xlabel('Trial (Round)', fontsize=12)
    axis.set_ylabel('Number of participants', fontsize=12)
    axis.tick_params(axis='x', labelrotation = 0) # xticks display vertically
    axis.set_axisbelow(True)
    axis.grid('on')
    handles, labels = axis.get_legend_handles_labels()
    axis.legend(reversed(handles), reversed(labels), title='Decision Type', loc='upper left', bbox_to_anchor=(1, 1))

In [16]:
from matplotlib.patches import Rectangle
plt.rcParams['hatch.linewidth'] = 0.5
def decision_type_heatmap(df, axis):
    '''
    Creates a heatmap of the decision types visualizing decision types per node per round.
    
    PARAMETERS:
    -----------
    df: Pandas DataFrame; contains information from a simulation run or experiment to plot.
    axis: matplotlib axis object; the axis on which to plot the data.
    
    RETURNS:
    --------
    None
    '''
    
    categories = ['BN', 'EC','RS', 'RP'] # three decision types NEW (pull from prior), RS (repeat self), RP (repeat partner)
    colors = ['xkcd:light red', 'xkcd:kelly green','xkcd:goldenrod', 'xkcd:cerulean'] # define colors for graph
    
    cmap = c.LinearSegmentedColormap.from_list("", colors, len(colors)) # using the same colors as other two plots for consistency.
    
    pivot_df = df.pivot(index='id', columns='TrialNumber', values='DecisionType') # creates the dataframe structure we want to display.
        # rows are specific agents, columns are the trial numbers, and each entry in the dataframe (array) is decision type.
    unique_decisions = ['BN', 'EC','RS', 'RP'] # these are the three unique decisions.
    response_to_num = {response: i for i, response in enumerate(unique_decisions)} # have to convert the data into numeric values to create the heatmap.
    num_df = pivot_df.applymap (lambda x: response_to_num[x] if pd.notna(x) else np.nan) # applies numeric mapping to entire dataframe we want to plot.
    
    g = sns.heatmap(num_df, fmt='', cmap=cmap, linewidths = 0.5, cbar_kws = dict(use_gridspec=False,location="bottom"), ax=axis) # creates a seaborn heatmap with specified colormap
    
    ## this is to add hatching over the boxes
    pts = df.groupby(['TrialNumber', 'id'])['Points'].sum()
    boxes = []
    for trial, node in pts.index:
        if pts[(trial, node)] == 1:
            boxes.append((trial-1, node-1))
    
    
    for pt in boxes:
        g.add_patch(Rectangle(pt, 1, 1, hatch='xxxxxx', fill=False, edgecolor='black', lw=0))
    
    axis.set_title('Decision Types by Trial and Node ID', fontsize=14)
    axis.set_xlabel('Trial (Round)', fontsize=12)
    axis.set_ylabel('Node ID', fontsize=12)
    
    # colorbar settings: creates a discreate colorbar that displays actual meanings of the color
    colorbar = axis.collections[0].colorbar
    colorbar.set_ticks([0.375, 1.125, 1.875, 2.625])
    colorbar.set_ticklabels(unique_decisions)
    
    black_box = Rectangle((1, 1), 1, 1, hatch='xxxxxx', fill=False, edgecolor='black', lw=0, label='Point Scored')
    axis.legend(handles=[black_box], loc='upper left', bbox_to_anchor=(1, 1))
    
    # visual tick parameters
    axis.tick_params(axis='x', labelrotation = 0)
    axis.tick_params(axis='y', labelrotation = 0)

In [17]:
def entropy_line_graph(df, axis):
    '''
    Creates a line/scatter plot of the entropy over the run as well as a line of best fit for this data.
    
    PARAMETERS:
    -----------
    df: Pandas DataFrame; contains information from a simulation run or experiment to plot.
    axis: matplotlib axis object; the axis on which to plot the data.
    
    RETURNS:
    --------
    None
    '''
    
    entropy = compute_entropy(df)
    axis.plot(range(1, len(entropy)+1), entropy, color = 'xkcd:frog green', alpha=0.5)
    axis.scatter(range(1, len(entropy)+1), entropy, s=50, color = 'xkcd:frog green', marker='o', label='Entropy Values')
    
    trials = df.TrialNumber.unique().tolist()
    z = np.polyfit(trials, entropy, 1)
    p = np.poly1d(z)
    axis.plot(trials, p(trials), color='xkcd:carnation', linewidth=4, ls=':', label='Least Squares Fit\n y=%.4fx+%.4f'%(z[0],z[1]))

    axis.legend()
    
    axis.set_title('Entropy per Trial', fontsize=14)
    axis.set_xlabel('Trial (Round)', fontsize=12)
    axis.set_ylabel('Entropy Value', fontsize=12)
    axis.set_axisbelow(True)
    axis.grid('on')

In [18]:
def response_heatmap(df, axis):
    '''
    Creates a heatmap of the actual responses, visualizing node responses per round.
    
    PARAMETERS:
    -----------
    df: Pandas DataFrame; contains information from a simulation run or experiment to plot.
    axis: matplotlib axis object; the axis on which to plot the data.
    
    RETURNS:
    --------
    None
    '''
    
    pivot_df = df.pivot(index='id', columns='TrialNumber', values='Response')
    unique_responses = pd.unique(pivot_df.to_numpy().flatten())
    response_to_num = {response: i for i, response in enumerate(unique_responses)}
    num_df = pivot_df.applymap (lambda x: response_to_num [x] if pd.notna(x) else np.nan)
    
    # Plotting
    sns.heatmap(num_df, fmt='', cmap='hsv', cbar=False, linewidths = 0.5, ax=axis)
    
    axis.set_title('Response Heatmap', fontsize=14)
    axis.set_xlabel('Trial (Round)', fontsize=12)
    axis.set_ylabel('Node ID', fontsize=12)
    axis.tick_params(axis='y', labelrotation = 0)

In [19]:
def most_common_responses(df, axis):
    '''
    Creates a line plot of the trends of the 5 most popular responses over time.
    
    PARAMETERS:
    -----------
    df: Pandas DataFrame; contains information from a simulation run or experiment to plot.
    axis: matplotlib axis object; the axis on which to plot the data.
    
    RETURNS:
    --------
    None
    '''
    
    subdf = df.groupby(['TrialNumber'])['Response'].value_counts()

    timedata = {r: [] for r in df.Response.unique()}

    for t in df.TrialNumber.unique():
        for response in timedata.keys():
            if (t, response) in subdf.index:
                timedata[response].append(subdf[(t, response)])
            else:
                timedata[response].append(0)
    
    maxresponsects = list(df.Response.value_counts().items())[:5]
    maxvals = [(ct, r) for r, ct in maxresponsects]
    
    colors = ['xkcd:dark seafoam', 'dodgerblue', 'mediumblue', 'mediumorchid', 'indigo']
    pltnums = []
    for i in range(len(maxvals)):
        pltnums += timedata[maxvals[i][1]]
        axis.plot(range(1, len(timedata[maxvals[i][1]])+1), timedata[maxvals[i][1]], alpha=0.5, color = colors[i])
        axis.scatter(range(1, len(timedata[maxvals[i][1]])+1), timedata[maxvals[i][1]], label=maxvals[i][1], color = colors[i])
    maxpt = max(pltnums) # this is just to set the tick values for the y axis.
    axis.legend(loc='upper left', bbox_to_anchor=(1, 1))
    axis.set_title('5 Most Popular Responses Over Time', fontsize=14)
    axis.set_xlabel('Trial (Round)', fontsize=12)
    axis.set_ylabel('Number of Participants Giving Response', fontsize=12)
    axis.set_axisbelow(True)
    axis.set_yticks(range(0, maxpt+3, 2))
    axis.tick_params(axis='x', labelrotation = 0)
    axis.grid('on')

In [20]:
def plot_transition_probs_bar(df, axis):
    '''
    Plots a bar graph to visualize the transition probabilities.
    
    PARAMETERS:
    -----------
    df: Pandas dataframe; contains information about the relevant simulation or experiment run.
    axis: matplotlib axis object; the acis on which to generate the plot.
    
    RETURNS:
    --------
    None
    '''
    dictvals = calc_transition_probs(df)
    
    ticklabels = [f'{k[0]}\n ↓ \n {k[1]}' for k in dictvals.keys()]
    
    axis.bar(range(len(dictvals)), dictvals.values())
    colors={'BN': 'xkcd:light red', 'EC':'xkcd:kelly green', 'RS': 'xkcd:goldenrod', 'RP': 'xkcd:cerulean'}
    x = 0
    for k, v in dictvals.items():
        axis.bar(x, v, color=colors[k[1]], label=f'transition to {k[1]}')
        axis.text(x, v+0.005, f'{round(v, 2)}', ha='center')
        x+=1
    
    # formatting for the legend
    handles, labels = axis.get_legend_handles_labels()
    handle_list, label_list = [], []
    for handle, label in zip(handles, labels):
        if label not in label_list:
            handle_list.append(handle)
            label_list.append(label)
            
    axis.legend(handle_list, label_list)

    axis.set_axisbelow(True)
    axis.grid('on')
    axis.set_xticks(range(len(ticklabels)), labels = ticklabels)
    axis.set_title('Transition Probabilities (Rounded)', fontsize=16)

In [21]:
def points_by_decision_type(df, axis):
    '''
    Creates a bar plot of number of points scored by decision type.
    
    PARAMETERS:
    -----------
    df: Pandas DataFrame; contains information about simulation or experiment run.
    axis: matplotlib axis object, on which to generate the plots.
    
    RETURNS:
    --------
    None
    '''
    
    colors={'BN': 'xkcd:light red', 'EC':'xkcd:kelly green', 'RS': 'xkcd:goldenrod', 'RP': 'xkcd:cerulean'}
    
    vals = df.groupby('DecisionType')['Points'].sum()
    
    ticklabels = [i for i in vals.index]
    
    x = 0
    for i in vals.index:
        axis.bar(x, vals[i], color=colors[i], label=i)
        axis.text(x, vals[i]+1, f'{int(vals[i])}', ha='center')
        x+=1
    
    axis.legend()
    axis.set_axisbelow(True)
    axis.grid('on')
    axis.set_xticks(range(4), labels = ticklabels)
    axis.set_title('Points by Decision Type', fontsize=16)

In [22]:
def create_all_plots(df, experiment, path, store):
    '''
    This function will generate 8 different plots for analysis of a simulation run. These plots will be:
        * A trial-level stacked bar graph which shows the proportion of participants that adopted each of the three strategies (NEW, RS, RP)
          at each timestep of the experiment/simulation.
        * A line plot which demonstrates the trends of each decision strategy (NEW, RS, RP) as the simulation goes on.
        * A heatmap plot which shows which nodes made what type of decision (NEW, RS, RP) at each timestep.
        * A line graph of the entropy per trial (round) and a line of best fit for this trend.
        * A response heatmap (which shows the distribution of actual choices on the network per timestep)
        * A line graph showing the trends of the 5 most popular responses over the duration of the experiment/simulation run.
        * A bar graph of transition probabilities.
        * A bar graph showing the number of points scored per decision strategy over the course of the whole experiment/simulation.
    
    PARAMETERS:
    -----------
    df: Pandas dataframe; produced by running the utility simulation. 
    experiment: string; name of experiment for the plot title.
    path: string; a filepath that will be used if we want to store the outputs. 
    store: bool; True if we want to store the outputs, False if not.
    
    RETURNS:
    --------
    This function does not return a value, simply produces plots.
    '''
    fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(nrows=4, ncols=2, figsize=(17.5, 30), layout='constrained') # create a figure with three subplots. 
        # figsize is large to account for runs of 100 node simulations because the heatmaps are hard to read.

                          
    fig.suptitle('Result Analysis Plots\n' + experiment, fontsize=16)
    
    ###############################################
    ### AXIS 1: DECISION TYPE STACKED BAR GRAPH ###
    ###############################################
    
    decision_type_stacked_bar(df, ax1)
    
    ##########################################
    ### AXIS 2: DECISION TYPE SCATTER PLOT ###
    ##########################################
    # This plot will better demonstrate the actual trends in decision types so we can analyze.
    
    decision_type_line(df, ax2)
    
    #####################################
    ### AXIS 3: DECISION TYPE HEATMAP ###
    #####################################
    # this visualizes which agents made which decisions at each timestep and allows for analysis of the network affects
    
    decision_type_heatmap(df, ax3)
    
    ############################################
    ### AXIS 4: ENTROPY PER ROUND LINE GRAPH ###
    ############################################
    
    entropy_line_graph(df, ax4)
    
    ###########################################
    ### AXIS 5: HEATMAP OF ACTUAL RESPONSES ###
    ###########################################
    
    response_heatmap(df, ax5)
    
    ##############################################################
    ### AXIS 6: FIVE MOST COMMON RESPONSES OVER TIME LINE PLOT ###
    ##############################################################
    
    most_common_responses(df, ax6)
    
    ##################################################
    ### AXIS 7: TRANSITION PROBABILITIES BAR GRAPH ###
    ##################################################
    
    plot_transition_probs_bar(df, ax7)
    
    #################################################
    ### AXIS 8: POINTS BY DECISION TYPE BAR GRAPH ###
    #################################################
    
    points_by_decision_type(df, ax8)
    
    
    #bc_stacked_bar(df, ax7)
    

    if store: # if store = True (this value is inherited from running the simulation) store the output as a png
              # could be called if we want to create the plots for an actual experimental run, in that case this parameter is not inherited.
        fig.savefig(f'{path}.png')
    
    return None